In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [41]:
c = feed.calendar
c['monday'] = 0
c['tuesday'] = 0
feed.calendar = c
trip_stats = feed.compute_trip_stats()
dates = feed.get_first_week()


In [54]:
#feed.compute_stop_stats(dates[:3])
feed.compute_route_stats(trip_stats, dates[:3])
#feed.compute_feed_stats(trip_stats, dates[:3])

Index(['route_id', 'route_short_name', 'route_type', 'num_trips',
       'num_trip_starts', 'num_trip_ends', 'is_loop', 'is_bidirectional',
       'start_time', 'end_time', 'max_headway', 'min_headway', 'mean_headway',
       'peak_num_trips', 'peak_start_time', 'peak_end_time',
       'service_distance', 'service_duration', 'service_speed',
       'mean_trip_distance', 'mean_trip_duration', 'date'],
      dtype='object')
Index(['route_id', 'route_short_name', 'route_type', 'num_trips',
       'num_trip_ends', 'num_trip_starts', 'is_bidirectional', 'is_loop',
       'start_time', 'end_time', 'max_headway', 'min_headway', 'mean_headway',
       'peak_num_trips', 'peak_start_time', 'peak_end_time',
       'service_duration', 'service_distance', 'service_speed',
       'mean_trip_distance', 'mean_trip_duration', 'date'],
      dtype='object')
Index(['route_id', 'route_short_name', 'route_type', 'num_trips',
       'num_trip_ends', 'num_trip_starts', 'is_bidirectional', 'is_loop',
       '

,date,end_time,is_bidirectional,is_loop,max_headway,mean_headway,mean_trip_distance,mean_trip_duration,min_headway,num_trip_ends,...,peak_end_time,peak_num_trips,peak_start_time,route_id,route_short_name,route_type,service_distance,service_duration,service_speed,start_time
0,20140526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20140527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20140528,24:02:00,1.0,0.0,35.0,29.955556,32.105476,0.972034,23.0,58.0,...,18:20:00,5.0,18:13:00,110-423,110,3.0,1894.223063,57.350000,33.029173,05:50:00
3,20140528,24:36:00,1.0,0.0,67.0,30.954545,34.528925,1.023276,25.0,57.0,...,08:05:00,5.0,07:57:00,111-423,111,3.0,2002.677623,59.350000,33.743515,06:02:00
4,20140528,22:31:00,0.0,1.0,60.0,60.000000,21.161459,0.600000,60.0,15.0,...,08:31:00,1.0,07:55:00,112-423,112,3.0,317.421880,9.000000,35.269098,07:55:00
5,20140528,18:42:00,1.0,0.0,60.0,60.000000,24.600261,0.655556,60.0,6.0,...,06:45:00,2.0,06:35:00,113-423,113,3.0,147.601565,3.933333,37.525822,06:05:00
6,20140528,22:23:00,1.0,0.0,60.0,60.000000,28.095473,0.832292,60.0,32.0,...,07:23:00,2.0,07:00:00,120-423,120,3.0,899.055146,26.633333,33.756764,05:34:00
7,20140528,23:51:00,0.0,0.0,NaN,NaN,40.502120,0.850000,NaN,2.0,...,22:51:00,1.0,22:00:00,120N-423,120N,3.0,81.004241,1.700000,47.649553,22:00:00
8,20140528,22:00:00,1.0,0.0,60.0,52.800000,17.237029,0.533333,30.0,34.0,...,07:48:00,3.0,07:46:00,121-423,121,3.0,586.058987,18.133333,32.319429,06:28:00
9,20140528,21:30:00,1.0,0.0,60.0,54.000000,16.420466,0.466667,30.0,33.0,...,07:30:00,2.0,07:16:00,122-423,122,3.0,541.875382,15.400000,35.186713,06:16:00


In [22]:
ts = feed.compute_route_time_series(trip_stats, dates[0], freq='6H')
# inds = ['num_trips', 'num_trip_ends', 'num_trip_starts']
# rids = ['110-423', '111-423', '112-423']

# ts.loc[:, (inds, rids)]

#ts = feed.compute_feed_time_series(trip_stats, [dates[0], dates[2]], freq='6H')
# ts = feed.compute_stop_time_series([dates[0], dates[2]], freq='6H')
# sid = '750015'
# ts.loc[:, ('num_trips', sid)]

ts

indicator           num_trip_ends                                           \
route_id                  110-423 111-423 112-423 113-423 120-423 120N-423   
2014-05-26 00:00:00             0       0       0       0       0        0   
2014-05-26 06:00:00            19      18       4       3      11        0   
2014-05-26 12:00:00            24      24       6       2      12        0   
2014-05-26 18:00:00            15      15       5       1       9        2   

indicator                                              ...    service_speed  \
route_id            121-423 122-423 123-423 130-423    ...          131-423   
2014-05-26 00:00:00       0       0       0       0    ...              NaN   
2014-05-26 06:00:00      13      13      21      11    ...        24.049902   
2014-05-26 12:00:00      13      12      24      12    ...        24.051157   
2014-05-26 18:00:00       8       8      14      10    ...        24.052642   

indicator                                                                   \
route_id              131N-423    133-423    140-423    141-423    142-423   
2014-05-26 00:00:00        NaN  29.675914  26.255898        NaN        NaN   
2014-05-26 06:00:00        NaN  24.905870  25.620285  20.802088  26.021598   
2014-05-26 12:00:00        NaN  24.743574  25.593499  20.809505  26.433521   
2014-05-26 18:00:00  24.067374  28.375748  26.830956  21.014223  26.385164   

indicator                                                       
route_id               143-423   143W-423    150-423  150E-423  
2014-05-26 00:00:00        NaN        NaN        NaN       NaN  
2014-05-26 06:00:00  24.366138        NaN  31.565566       NaN  
2014-05-26 12:00:00  24.377598        NaN  31.524480       NaN  
2014-05-26 18:00:00  23.786714  30.331218  31.702986  34.12138  

[4 rows x 120 columns]

In [23]:
ts.columns

MultiIndex(levels=[['num_trip_ends', 'num_trip_starts', 'num_trips', 'service_distance', 'service_duration', 'service_speed'], ['110-423', '111-423', '112-423', '113-423', '120-423', '120N-423', '121-423', '122-423', '123-423', '130-423', '131-423', '131N-423', '133-423', '140-423', '141-423', '142-423', '143-423', '143W-423', '150-423', '150E-423']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

In [7]:
dts = gt.downsample(ts, freq='D')
#dts.loc[:, (inds, rids)]
#dts.loc[:, ('num_trips', sid)]
dts

,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
2014-05-26,617.0,622.0,622.0,13774.027234,472.6,29.145212
2014-05-27,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-28,617.0,622.0,622.0,13774.027234,472.6,29.145212


In [26]:
f = feed.compute_feed_stats(trip_stats, '2001')
f


""


In [25]:
f.date

Series([], Name: date, dtype: object)

In [44]:
feed.compute_stop_stats(['20140601', '20140603'])

,stop_id,num_routes,num_trips,start_time,end_time,max_headway,min_headway,mean_headway,date
0,750000,1.0,16.0,07:16:00,22:16:00,60.0,60.0,60.000000,20140601
1,750001,1.0,16.0,07:18:00,22:18:00,60.0,60.0,60.000000,20140601
2,750002,1.0,16.0,07:20:00,22:20:00,60.0,60.0,60.000000,20140601
3,750003,1.0,16.0,07:21:00,22:21:00,60.0,60.0,60.000000,20140601
4,750004,1.0,16.0,07:23:00,22:23:00,60.0,60.0,60.000000,20140601
5,750005,1.0,16.0,07:25:00,22:25:00,60.0,60.0,60.000000,20140601
6,750006,1.0,16.0,07:26:00,22:26:00,60.0,60.0,60.000000,20140601
7,750007,1.0,16.0,07:27:00,22:27:00,60.0,60.0,60.000000,20140601
8,750008,1.0,16.0,07:28:00,22:28:00,60.0,60.0,60.000000,20140601
9,750009,1.0,16.0,07:29:00,22:29:00,60.0,60.0,60.000000,20140601


In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()